# Förberedelser

## Installera

In [ ]:
!pip install sparqlwrapper

## Importera

In [ ]:
import SPARQLWrapper as sparql
import pandas as pd
import html

## Välj Libris-miljö

In [ ]:
#libris = "https://libris.kb.se/sparql"
#libris = "https://libris-qa.kb.se/sparql"
libris = "https://libris-dev2.kb.se/sparql"

# Förändringar

## Verk

### Verkstyp
Det finns en helt ny uppsättning värden att välja på för egenskapen Verkstyp

In [ ]:
# TODO Kommer de "gamla" verkstyperna (Text mfl) finnas kvar, och hur utesluter man isåfall dem i en SPARQL-query?

work_type_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT * WHERE {
    ?workType rdfs:subClassOf/(rdfs:subClassOf?) :Work .
    OPTIONAL { ?workType :label ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?workType :label ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    OPTIONAL { ?workType :category ?classCategory . }
    OPTIONAL { ?workType :abstract ?abstract . }

    }
    ORDER BY ?classCategory ?workType

  '''

work_types = sparql.get_sparql_dataframe(endpoint=libris, query=work_type_query)

print("Totalt antal:", work_types.shape[0])

work_types.head(35)

Totalt antal: 31


,workType,enLabel,svLabel,classCategory
0,https://id.kb.se/vocab/Audio,Audio,Ljud,NaN
1,https://id.kb.se/vocab/Cartography,Cartography,Karta,NaN
2,https://id.kb.se/vocab/Collection,Collection,Samling,NaN
3,https://id.kb.se/vocab/Integrating,Integrating resource,Integrerande resurs,NaN
4,https://id.kb.se/vocab/Integrating,Integrating,Integrerande resurs,NaN
5,https://id.kb.se/vocab/Kit,Kit,Paket,NaN
6,https://id.kb.se/vocab/ManuscriptCartography,Manuscript Cartography,Karthandskrift,NaN
7,https://id.kb.se/vocab/ManuscriptNotatedMusic,Manuscript Notated Music,Musikhandskrift,NaN
8,https://id.kb.se/vocab/ManuscriptText,Manuscript Text,Texthandskrift,NaN
9,https://id.kb.se/vocab/MixedMaterial,Mixed material,Blandat material,NaN


### Verkskategori
Egenskapen Genre/Form (verk) byter namn till Kategori (verk). Termlistorna som kan användas i egenskapen förfinas och förbättras.


#### Fler betydelsefulla relationer
De värden som går att lägga till under Kategori (verk) är sammanlänkade med smalare/bredare-relationer. Dessa relationer möjliggör automatisk härledning och navigation mellan termerna.

In [ ]:
work_categories_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT ?workCategory ?categoryLabel ?oneLevelUpLabel ?twoLevelsUpLabel
  WHERE {
    ?workCategory :inScheme <https://id.kb.se/term/saogf> ;
                  :prefLabel ?categoryLabel .
    OPTIONAL {
      ?workCategory :broader ?oneLevelUp .
      ?oneLevelUp :prefLabel ?oneLevelUpLabel .
      OPTIONAL { ?oneLevelUp :broader ?twoLevelsUp . ?twoLevelsUp :prefLabel ?twoLevelsUpLabel .}
      }
    }

  '''

work_categories = sparql.get_sparql_dataframe(endpoint=libris, query=work_categories_query)

print("Totalt antal:", work_categories.shape[0])

work_categories.head(20)

Totalt antal: 3594


,workCategory,categoryLabel,oneLevelUpLabel,twoLevelsUpLabel
0,https://id.kb.se/term/saogf/Serieromaner,Serieromaner,Romaner,Skönlitteratur
1,https://id.kb.se/term/saogf/Dagboksromaner,Dagboksromaner,Romaner,Skönlitteratur
2,https://id.kb.se/term/saogf/Utvecklingsromaner,Utvecklingsromaner,Romaner,Skönlitteratur
3,https://id.kb.se/term/saogf/Pikareskromaner,Pikareskromaner,Romaner,Skönlitteratur
4,https://id.kb.se/term/saogf/Kortromaner,Kortromaner,Romaner,Skönlitteratur
5,https://id.kb.se/term/saogf/Nyckelromaner,Nyckelromaner,Romaner,Skönlitteratur
6,https://id.kb.se/term/saogf/Brevromaner,Brevromaner,Romaner,Skönlitteratur
7,https://id.kb.se/term/saogf/Kollektivromaner,Kollektivromaner,Romaner,Skönlitteratur
8,https://id.kb.se/term/saogf/Urban%20fantasy,Urban fantasy,Fantasy,Skönlitteratur
9,https://id.kb.se/term/saogf/Bibliska%20ber%C3%...,Bibliska berättelser,Religiösa skildringar,Skönlitteratur


#### Find

Det finns en delmängd av kategorier som vi identifierat som särksilt användbara för att gruppera verk på en hög nivå. Dessa utgör delsamlingen *find*. Vissa av termerna i *find*-samlingen är även "topptermer" i kategoriträdet.


In [ ]:
work_find_categories_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT * WHERE {
    ?findCategory :inCollection <https://id.kb.se/term/div/find> .
    OPTIONAL { ?findCategory :singularLabel ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?findCategory :singularLabel ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    }
    ORDER BY ?findCategory

  '''

work_find_categories = sparql.get_sparql_dataframe(endpoint=libris, query=work_find_categories_query)

print("Totalt antal:", work_find_categories.shape[0])

work_find_categories.head(50)

Totalt antal: 7


,findCategory,svLabel,enLabel
0,https://id.kb.se/term/ktg/Cartography,Kartografiskt material,NaN
1,https://id.kb.se/term/ktg/Literature,Litteratur,Literature
2,https://id.kb.se/term/ktg/MovingImage,NaN,NaN
3,https://id.kb.se/term/ktg/Music,NaN,NaN
4,https://id.kb.se/term/ktg/Object,NaN,NaN
5,https://id.kb.se/term/ktg/Software,NaN,NaN
6,https://id.kb.se/term/ktg/StillImage,NaN,NaN


#### Identify

Det finns en delmängd av kategorier som vi identifierat som särksilt användbara för att på en lagom allmän men ändå specifik nivå säga vad "vad något är". Dessa utgör delsamlingen *identify*.

In [ ]:
work_identify_categories_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT * WHERE {
    ?identifyCategory :inCollection <https://id.kb.se/term/div/identify> .
    OPTIONAL { ?identifyCategory :singularLabel ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?identifyCategory :singularLabel ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    }
    ORDER BY ?identifyCategory

  '''

work_identify_categories = sparql.get_sparql_dataframe(endpoint=libris, query=work_identify_categories_query)

print("Antal typer:", work_identify_categories.shape[0])

work_identify_categories.head(50)

Antal typer: 32


,identifyCategory,enLabel,svLabel
0,https://id.kb.se/term/ktg/Audiobook,Audiobook,Ljudbok
1,https://id.kb.se/term/ktg/Biography,Biography,Biografi
2,https://id.kb.se/term/ktg/ChapterBook,Chapter Book,Kapitelbok
3,https://id.kb.se/term/ktg/ComputerGame,Computer Game,Datorspel
4,https://id.kb.se/term/ktg/Cookbook,Cookbook,Kokbok
5,https://id.kb.se/term/ktg/Database,Database,Databas
6,https://id.kb.se/term/ktg/Drama,NaN,NaN
7,https://id.kb.se/term/ktg/Ephemera,NaN,NaN
8,https://id.kb.se/term/ktg/Essay,NaN,NaN
9,https://id.kb.se/term/ktg/FictionLiterature,NaN,NaN


## Instans

### Instanstyp

Det finns en helt ny uppsättning värden att välja på för egenskapen Instanstyp

In [ ]:
instance_type_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT * WHERE {
    ?instanceType rdfs:subClassOf/(rdfs:subClassOf?) :Instance .
    OPTIONAL { ?instanceType :label ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?instanceType :label ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    OPTIONAL { ?instanceType :category ?classCategory . }
    }
    ORDER BY ?classCategory ?instanceType

  '''

instance_types = sparql.get_sparql_dataframe(endpoint=libris, query=instance_type_query)

print("Totalt antal:", instance_types.shape[0])

instance_types.head(20)

Totalt antal: 19


,instanceType,enLabel,svLabel,classCategory
0,https://id.kb.se/vocab/Archival,Archival controlled,Arkiv,NaN
1,https://id.kb.se/vocab/DigitalResource,Digital Resource,Digital resurs,NaN
2,https://id.kb.se/vocab/Electronic,Electronic,Elektronisk,NaN
3,https://id.kb.se/vocab/Globe,Globe,Kartglob,NaN
4,https://id.kb.se/vocab/Manuscript,Manuscript,Handskrift,NaN
5,https://id.kb.se/vocab/Map,Map,Karta,NaN
6,https://id.kb.se/vocab/Microform,Microform,Mikroform,NaN
7,https://id.kb.se/vocab/PhysicalResource,Physical resource,Fysisk resurs,NaN
8,https://id.kb.se/vocab/Print,Printed,Tryck,NaN
9,https://id.kb.se/vocab/SoundRecording,Sound Recording,Ljudinspelning,NaN


### Instanskategori

Egenskapen carrier byter namn till Kategori (instans).

De värden som går att lägga till under Kategori (instans) är sammanlänkade med smalare/bredare-relation

In [ ]:
# TODO Hur identifierar vi kategorier som kan användas på instanser?
# Oavsett om de är carrierTypes, mediaTypes, eller BF-typer.

instance_carrierTypes_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT ?mediaType ?carrierType ?enLabel ?svLabel WHERE {
    ?carrierType a :CarrierType ;
      :broader ?mediaType .
    OPTIONAL { ?carrierType :label ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?carrierType :label ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    }
    ORDER BY DESC(?mediaType) ?carrierType

  '''

instance_carrier_types = sparql.get_sparql_dataframe(endpoint=libris, query=instance_carrierTypes_query)

print("Antal typer:", instance_carrier_types.shape[0])

instance_carrier_types.head(20)

Antal typer: 79


,mediaType,carrierType,enLabel,svLabel
0,https://id.kb.se/term/rda/Video,https://id.kb.se/term/rda/VideoCartridge,Video cartridge,Videomagasin
1,https://id.kb.se/term/rda/Video,https://id.kb.se/term/rda/Videocassette,Videocassette,Videokassett
2,https://id.kb.se/term/rda/Video,https://id.kb.se/term/rda/Videodisc,Video Disc,Videoskiva
3,https://id.kb.se/term/rda/Video,https://id.kb.se/term/rda/VideotapeReel,Videotape reel,Videospole
4,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Card,Card,Bildkort
5,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Flipchart,Flipchart,Blädderblock
6,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Object,NaN,NaN
7,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Roll,Roll,Rulle
8,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Sheet,Sheet,Ark
9,https://id.kb.se/term/rda/Unmediated,https://id.kb.se/term/rda/Volume,Volume,Volym


### Utgivningssätt

Det finns en helt ny uppsättning värden att välja på för egenskapen Utgivningssätt


In [ ]:
instance_issuanceTypes_query = '''
  prefix : <https://id.kb.se/vocab/>

  SELECT * WHERE {
    ?issuanceType a :IssuanceType .
    OPTIONAL { ?issuanceType :label ?enLabel . FILTER (LANG(?enLabel) = "en") }
    OPTIONAL { ?issuanceType :label ?svLabel . FILTER (LANG(?svLabel) = "sv") }
    OPTIONAL { ?issuanceType :category ?classCategory . }
    }
    ORDER BY ?classCategory ?issuanceType

  '''

instance_issuance_types = sparql.get_sparql_dataframe(endpoint=libris, query=instance_issuanceTypes_query)

print("Antal typer:", instance_issuance_types.shape[0])

instance_issuance_types.head(50)

Antal typer: 2


,issuanceType,svLabel
0,https://id.kb.se/vocab/MultipleUnits,Flera enheter
1,https://id.kb.se/vocab/SingleUnit,Enskild enhet
